In [1]:
import json
import sys
import os

from easydict import EasyDict as edict
import pymysql as sql

## 하위 폴더에 있는 패키지 로딩을 위한 밑작업
SEP       = os.path.sep
MISC_PATH = SEP.join(os.getcwd().split(SEP)[:-2])
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-4])
sys.path.append(MISC_PATH)

from misc.config import *

[INFO] config.json 파일을 로딩합니다.
[INFO] ports.json 파일을 로딩합니다.


In [2]:
host   = CONFIGS.global_host
port   = PORTS.sql_port
user   = CONFIGS.sql_user
passwd = CONFIGS.sql_passwd

## 노트북 재시작할때마다 DB삭제용 플래그
not_first = True

In [3]:
## DB 서버에 연결
conn   = sql.connect(host     = host, user = user, port = port,
                     password = passwd)

cursor = conn.cursor()

In [4]:
def print_results(text: str, results: tuple):
    
    print(f'{text}')
    for result in results: print(result)
    print('\n')

In [5]:
query = 'show databases'
cursor.execute(query)
print_results('[before]', cursor.fetchall())

## 생성하고자 하는 DB가 없는 경우에만 생성
cursor.execute('create database if not exists study')
cursor.execute('create database if not exists dummy')

cursor.execute(query)
print_results('[middle]', cursor.fetchall())

## DB 제거
cursor.execute('drop database dummy')
cursor.execute(query)
print_results('[after]', cursor.fetchall())

[before]
('DoveNest',)
('information_schema',)
('mysql',)
('performance_schema',)
('study',)
('sys',)


[middle]
('DoveNest',)
('dummy',)
('information_schema',)
('mysql',)
('performance_schema',)
('study',)
('sys',)


[after]
('DoveNest',)
('information_schema',)
('mysql',)
('performance_schema',)
('study',)
('sys',)




In [6]:
## 사용하고자 하는 DB 선택
cursor.execute('use study')

cursor.execute('drop table if exists tCity')
cursor.execute('drop table if exists tStaff')

0

In [7]:
query = 'show tables'
cursor.execute(query)
print_results('[before]', cursor.fetchall())

[before]
('department',)
('dept_',)
('employee',)
('employee_',)
('tCity_bu',)
('tReport',)
('tStaff_bu',)
('tSudo',)




In [8]:
## 마찬가지로 생성하고자 하는 이름의 테이블이 없을때만 테이블 생성
crud_query = '''create table if not exists tCity
                (
                 NAME VARCHAR(20) PRIMARY KEY,
                 AREA INTEGER NOT NULL,
                 POPU INTEGER NOT NULL,
                 METRO VARCHAR(5) NOT NULL,
                 REGION VARCHAR(10) NOT NULL)
             '''

dummy_query = '''
                create table if not exists dummy (idx INTEGER)
              '''

cursor.execute(crud_query)
cursor.execute(dummy_query)
cursor.execute(query)
print_results('[middle]', cursor.fetchall())

## 테이블 제거
cursor.execute('drop table dummy')
cursor.execute(query)
print_results('[after]', cursor.fetchall())

[middle]
('department',)
('dept_',)
('dummy',)
('employee',)
('employee_',)
('tCity',)
('tCity_bu',)
('tReport',)
('tStaff_bu',)
('tSudo',)


[after]
('department',)
('dept_',)
('employee',)
('employee_',)
('tCity',)
('tCity_bu',)
('tReport',)
('tStaff_bu',)
('tSudo',)




In [9]:
## 데이터 추가하는 쿼리함수
def insert(columns: list, values: list, table_name: str = 'tCity'):
    
    columns = ', '.join(columns)
    v       = ', '.join(['%s' for _ in values])
    query   = f'insert into {table_name} ({columns}) values ({v})'
    
    cursor.execute(query, values)

## **1. 데이터 조회 (SELECT 문)**
~~~ SQL
    SELECT '필드명' FROM '테이블 이름' [WHERE 조건][ORDER BY 정렬기준];
~~~
- []안에 있는 옵션들은 생략가능하다.
- 필드명에 '*'을 입력하면 모든 필드의 데이터를 가져올 수 있다.
    - 모든 필드의 데이터를 가져올 수는 있지만, 출력 결과가 가변적이라  
      위험한 경우도 있다.  
   - 코드가 불명확해지고, 어플리케이션의 메모리가 낭비되어 지양하는 것이 좋다. 
          

In [10]:
## 데이터 조회하는 쿼리함수
def select(text  :str = None, column:str         =  '*', table_name   : str = 'tCity', limit_k = None,
           cond  :str = None, order_column : str = None, order = 'asc', distinct = False):
    
    distinct = 'distinct' if distinct else '' 
    query    = f'select {distinct} {column} from {table_name} '
    
    ## 조회하는데 조건이 있을 경우에 where문 추가
    if cond != None: query += f'where {cond}'
    
    ## 조회하는데 정렬하고자 하는 컬럼이 있다면 order by문 추가
    if order_column != None: query += f'order by {order_column} {order}'
    
    ## 조회하는 레코드 개수 설정하는 LIMIT 문 추가
    if limit_k != None: query += f' limit {limit_k}'
    
    print(f'[query] {query};\n')
    cursor.execute(query)
    if text != None: print_results(text, cursor.fetchall())

In [11]:
select('[before]')

column = ['NAME', 'AREA', 'POPU', 'METRO', 'REGION']
values = [
          ['부산',    765,    342,     'y',    '경상'],
          ['서울',    605,    974,     'y',    '경기'],
          ['순천',    910,     27,     'n',    '전라'],
          ['오산',     42,     21,     'n',    '경기'],
          ['전주',    205,     65,     'n',    '전라'],
          ['청주',    940,     83,     'n',    '충청'],
          ['춘천',   1116,     27,     'n',    '강원'],
          ['홍천',   1819,      7,     'n',    '강원'],
         ]

for value in values: insert(column, value)

select('[after]')

[query] select  * from tCity ;

[before]


[query] select  * from tCity ;

[after]
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('오산', 42, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')




In [12]:
select(    '[order test]',       'REGION, POPU', order_column = 'POPU', order = 'desc')
select('[condition test]', cond = 'METRO = "n"', order_column = 'POPU')

[query] select  REGION, POPU from tCity order by POPU desc;

[order test]
('경기', 974)
('경상', 342)
('충청', 83)
('전라', 65)
('전라', 27)
('강원', 27)
('경기', 21)
('강원', 7)


[query] select  * from tCity where METRO = "n"order by POPU asc;

[condition test]
('홍천', 1819, 7, 'n', '강원')
('오산', 42, 21, 'n', '경기')
('순천', 910, 27, 'n', '전라')
('춘천', 1116, 27, 'n', '강원')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')




In [13]:
## 다른 예제를 위한 새로운 테이블 생성
query = '''
            create table if not exists tStaff
            (name   CHAR(15) PRIMARY KEY,
             depart CHAR(10) NOT NULL,
             gender CHAR(3)  NOT NULL,
             joindate DATE NOT NULL,
             grade CHAR(10) NOT NULL,
             salary INT NOT NULL,
             score DECIMAL(5, 2) NULL)
        '''

cursor.execute(query)

0

In [14]:
column = ['name', 'depart', 'gender', 'joindate', 'grade', 'salary', 'score']
values = [
            ('김유신' , '총무부','남','2000-2-3','이사',420,88.8),
            ('유관순' , '영업부','여','2009-3-1','과장',380, None),
            ('안중근' , '인사과','남','2012-5-5','대리',256,76.5),
            ('윤봉길' , '영업부','남','2015-8-15','과장',350,71.25),
            ('강감찬' , '영업부','남','2018-10-9','사원',320,56.0),
            ('정몽주' , '총무부','남','2010-9-16','대리',370,89.5),
            ('허난설헌', '인사과','여','2020-1-5','사원',285,44.5),
            ('신사임당', '영업부','여','2013-6-19','부장',400,92.0),
            ('성삼문' , '영업부','남','2014-6-8','대리',285,87.75),
            ('논개'   , '인사과','여','2010-9-16','대리',340,46.2),
            ('황진이'  , '인사과','여','2012-5-5','사원',275,52.5),
            ('이율곡'  , '총무부','남','2016-3-8','과장',385,65.4),
            ('이사부'  , '총무부','남','2000-2-3','대리',375,50),
            ('안창호'  , '영업부','남','2015-8-15','사원',370,74.2),
            ('을지문덕' , '영업부','남','2019-6-29','사원',330, None),
            ('정약용'  , '총무부','남','2020-3-14','과장',380,69.8),
            ('홍길동'  , '인사과','남','2019-8-8','차장',380,77.7),
            ('대조영'  , '총무부','남','2020-7-7','차장',290,49.9),
            ('장보고'  , '인사과','남','2005-4-1','부장',440,58.3),
            ('선덕여왕' , '인사과','여','2017-8-3','사원',315,45.1),
         ]


for value in values: insert(column, value, table_name = 'tStaff')

select('[tStaff table]', table_name = 'tStaff')

[query] select  * from tStaff ;

[tStaff table]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 400, Decimal('92.00'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 380, None)
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 350, Decimal('71.25'))
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), '사원', 330, None)
('이사부', '총무부', '남', datetime.date(2000, 2, 3), '대리', 375

##### **연습 문제 #001.**

> Q. tStaff 테이블에서 이름과 부서, 직급 필드만 출력하라


In [15]:
select('[A.]', 'name, depart, grade', table_name = 'tStaff')

[query] select  name, depart, grade from tStaff ;

[A.]
('강감찬', '영업부', '사원')
('김유신', '총무부', '이사')
('논개', '인사과', '대리')
('대조영', '총무부', '차장')
('선덕여왕', '인사과', '사원')
('성삼문', '영업부', '대리')
('신사임당', '영업부', '부장')
('안중근', '인사과', '대리')
('안창호', '영업부', '사원')
('유관순', '영업부', '과장')
('윤봉길', '영업부', '과장')
('을지문덕', '영업부', '사원')
('이사부', '총무부', '대리')
('이율곡', '총무부', '과장')
('장보고', '인사과', '부장')
('정몽주', '총무부', '대리')
('정약용', '총무부', '과장')
('허난설헌', '인사과', '사원')
('홍길동', '인사과', '차장')
('황진이', '인사과', '사원')




- SELECT 문이 출력하는 결과를 결과 셋(result set) 또는  
  로우 셋(row set) 이라고 하는 테이블이 생성된다.
  - 결과 셋의 필드명은 테이블에서 정의한 필드명과 동일하다.  
  
 ### **별명 (AS)**
 
- SELECT 문의 필드명 뒤에 AS를 사용해 별명을 설정하면,  
  출력 결과에서 설정한 별명으로 확인할 수 있다.
  - 공백이나 특수문자 포함이 가능하다.

In [16]:
select(column = 'name as 도시명, popu as "인구(만 명)"')

[query] select  name as 도시명, popu as "인구(만 명)" from tCity ;



![AS 예제](../../../assets/SQL/CH4/AS_EXAMPLE.jpeg)

##### **연습 문제 #002.** 

> Q. 결과창의 헤더에 다음과 같이 출력하도록 하는 필드 목록을 작성하라.


||도시|인구(만명)|지역|
|---|---|---|---|
|1|서울|947|경기|
|2|부산|342|경상|
|3|오산|21|경기|
|4|청주|83|충청|

![연습문제 2번](../../../assets/SQL/CH4/EXER_#002.jpeg)

#### 계산값 출력
- SELECT 이후 필드명에 계산식을 이용하면 테이블에 저장된 값을 가공할 수 있다.
![계산식 적용](../../../assets/SQL/CH4/Calc.jpeg)


- 계산식을 이용하면 기존 테이블에 존재하지 않는 정보도 추가 할 수 있다.
![계산식 적용2](../../../assets/SQL/CH4/Calc2.jpeg)

- SELECT 문을 아래 둘 중 하나의 방법으로 간단한 계산기로도 사용할 수 있다.
    ~~~ SQL
        - select [계산식] from dual;  
        - select [계산식];
    ~~~

In [17]:
cursor.execute('select 60 * 60 * 24 as day from dual')
print_results('[with from]', cursor.fetchall())
cursor.execute('select 60 * 60 * 24 as day')
print_results('[without from]', cursor.fetchall())

[with from]
(86400,)


[without from]
(86400,)




##### **연습 문제 #003.** 

> Q. SELECT 문을 사용해 1년은 몇 초인지 계산하라.


In [18]:
cursor.execute('select 60 * 60 * 24 *365')
print_results('[A.]', cursor.fetchall())

[A.]
(31536000,)




## **2. 조건문**
### 2-1. 필드 비교
- WHERE 절은 읽을 레코드의 조건을 지정해준다.
    - SELECT는 조회가 주 기능이어서 주로 WHERE절과 사용한다.
    - WHERE 절은 DELETE, UPDATE 등의 명령과도 함께 사용된다.
    
- WHERE 절에서 조건에 사용할 수 있는 연산자는 아래와 같다.
    - 필드의 자료형과 비교대상의 자료형이 호환되어야 한다.  
    
- 숫자는 상수를 그대로 쓰지만, 문자열과 날짜 상수는 따옴표로 감싸야한다.
    
||연산자|설명|예|
|---|---|---|---|
|1|A = B|A와 B가 같다.|WHERE name='서울'|
|2|A <> B, A !=B|A와 B가 같지 않다.|WHERE region <> '경기'|
|3|A > B|A가 B보다 크다.|WHERE popu > 300|
|4|A < B|A가 B보다 작다.|WHERE popu < 200|
|5|A >= B|A가 B보다 크거나 같다.|WHERE area >= 1000|
|6|A <= B|A가 B보다 작거나 같다.|WHERE area <= 100|


In [19]:
select('[name equal to "서울"]'               , cond = 'name = "서울"')
select('[region not equal to "경기"]'         , cond = 'region <> "경기"')
select('[popu greater than 300]'             , cond = 'popu > 300')
select('[popu less than 200]'                , cond = 'popu < 200')
select('[area greater than or equal to 3000]', cond = 'area >= 1000')
select('[area less than or equal to 100]'    , cond = 'area <= 100')

[query] select  * from tCity where name = "서울";

[name equal to "서울"]
('서울', 605, 974, 'y', '경기')


[query] select  * from tCity where region <> "경기";

[region not equal to "경기"]
('부산', 765, 342, 'y', '경상')
('순천', 910, 27, 'n', '전라')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  * from tCity where popu > 300;

[popu greater than 300]
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')


[query] select  * from tCity where popu < 200;

[popu less than 200]
('순천', 910, 27, 'n', '전라')
('오산', 42, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  * from tCity where area >= 1000;

[area greater than or equal to 3000]
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  * from tCity where area <= 100;

[area less than or equal to 100]
('오산', 42, 21, 'n', '경기')




##### **연습 문제 #004.** 

> Q1. 인구가 10만명 미만인 도시의 이름을 출력하라.  
> Q2. 전라도에 있는 도시의 정보를 출력하라.  
> Q3. 월급이 400만원 이상인 직원의 이름을 출력하라.


In [20]:
select('[A1.]', column = 'name', cond = 'popu * 10000 > 100000')
select('[A2.]', cond = 'region = "전라"')
select('[A3.]', column = 'name', table_name = 'tStaff', cond = 'salary * 10000 >= 4000000')

[query] select  name from tCity where popu * 10000 > 100000;

[A1.]
('부산',)
('서울',)
('순천',)
('오산',)
('전주',)
('청주',)
('춘천',)


[query] select  * from tCity where region = "전라";

[A2.]
('순천', 910, 27, 'n', '전라')
('전주', 205, 65, 'n', '전라')


[query] select  name from tStaff where salary * 10000 >= 4000000;

[A3.]
('김유신',)
('신사임당',)
('장보고',)




### 2-2. NULL 비교
- NULL은 값이 입력되어 있지 않은 상태를 말한다.
    - 값이 지정되어 있지 않기 때문에, 0이나 빈 문자열과도 다른 상태이다.
    
> #### <e.g.>
~~~ SQL  
    CREATE TABLE tCity
    (
        name CHAR(10) PRIMARY KEY,
        area INT NULL,
        popu INT NULL,
        metro CHAR(1) NOT NULL,
        region CHAR(6) NOT NULL
    );
~~~

- 선언문 뒤에 NULL이 있으면 값을 입력하지 않아도 된다는 뜻이다.
    - 위 예에서는 인구와 면적은 값을 입력하지 않아도 되고, 그 외 데이터 들은 입력해야한다.
    - NULL 비교를 위해서는 연산자로 is를 사용한다.

In [21]:
## tStaff 테이블에서 score가 NULL인 직원들만 추출
select('[score is NULL in tStaff]', table_name = 'tStaff', cond = f'score is NULL')

[query] select  * from tStaff where score is NULL;

[score is NULL in tStaff]
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 380, None)
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), '사원', 330, None)




In [22]:
select('[score is not NULL in tStaff]', table_name = 'tStaff', cond = f'score is not NULL')

[query] select  * from tStaff where score is not NULL;

[score is not NULL in tStaff]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 400, Decimal('92.00'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 350, Decimal('71.25'))
('이사부', '총무부', '남', datetime.date(2000, 2, 3), '대리', 375, Decimal('50.00'))
('이율곡', '총무부', '남', datetime.date(2016, 3, 8), '과장', 385, Decimal('65.40

### 2-3. 논리 연산자
- 두 가지 이상의 조건을 동시에 점검할 때는 AND, OR 논리 연산자를 사용한다.
    - A AND B | A와 B 두 조건을 모두 만족하는 데이터
    - A OR  B | A와 B 둘 중 하나의 조건만 만족하는 데이터
    - AND 연산자가 OR 연산자보다 우선 순위가 높고, 괄호로 우선 순위를 지정해 줄 수 있다.
    
- 조건에 부합하지 않는 데이터를 구하기 위해서는 NOT 논리 연산자를 사용한다.
    - "!="와 같은 기능을 하지만, 복합 조건의 부정을 취할 때는 NOT을 사용하면 편하다.

In [23]:
select('[지역이 경기도면서 인구가 50만 이상이거나 지역이 경기도가 아니여도 면적이 500 이상]', cond = 'region = "경기" and popu >= 50 or area >= 500')
select('[지역이 경기도면서, 인구가 50만이 넘거나 면적이 500 이상]', cond = 'region = "경기" and (popu >= 50 or area >= 500)')
select('[지역이 경기도가 아닌 데이터]', cond = 'region != "경기"')
select('[지역이 경기도가 아닌 데이터]', cond = 'NOT (region = "경기")')

[query] select  * from tCity where region = "경기" and popu >= 50 or area >= 500;

[지역이 경기도면서 인구가 50만 이상이거나 지역이 경기도가 아니여도 면적이 500 이상]
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  * from tCity where region = "경기" and (popu >= 50 or area >= 500);

[지역이 경기도면서, 인구가 50만이 넘거나 면적이 500 이상]
('서울', 605, 974, 'y', '경기')


[query] select  * from tCity where region != "경기";

[지역이 경기도가 아닌 데이터]
('부산', 765, 342, 'y', '경상')
('순천', 910, 27, 'n', '전라')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  * from tCity where NOT (region = "경기");

[지역이 경기도가 아닌 데이터]
('부산', 765, 342, 'y', '경상')
('순천', 910, 27, 'n', '전라')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')




##### **연습 문제 #005.** 

> Q1. 직원 목록에서 월급이 300 미만이면서 성취도는 60 이상인 직원이 누구인지 조사하라.  
> Q2. 영업부의 여직원 이름을 조사하라.


In [24]:
select('[A1.]', table_name = 'tStaff', cond = 'salary < 300 and score >= 60')
select('[A2.]', table_name = 'tStaff', column = 'name', cond = 'gender = "여"')

[query] select  * from tStaff where salary < 300 and score >= 60;

[A1.]
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))


[query] select  name from tStaff where gender = "여";

[A2.]
('논개',)
('선덕여왕',)
('신사임당',)
('유관순',)
('허난설헌',)
('황진이',)




### 2-4. LIKE
- LIKE는 "=" 연산자와 달리 패턴으로 부분 문자열을 검색한다.
- LIKE 문의 패턴에는 아래와 같은 와일드 카드를 사용한다.  

||와일드 카드|설명|
|---|---|---|
|1|%|복수개의 문자와 대응. DoS의 *와 동일한 의미를 가지며 %자리에 임의 개수의 임의 문자가 올 수 있다.|
|2|_|하나의 문자와 대응. DoS의 ?와 동일한 의미를 가지며 _자리에 하나의 임의 문자가 올 수 있다.|
|3|[]|[] 안에 포함된 문자 리스트 중 하나의 문자와 대응한다.|
|4|[^]|[^] 안에 포함된 문자 리스트에 포함되지 않은 하나의 문자와 대응한다.|

In [25]:
## 도시 이름에 '천'이 들어가는 도시 조회
## 아래 조건식은 앞뒤의 문자열이 얼마나 길던지 문자열에 "천"이 들어가는 데이터들을 모두 출력해준다.
select('["천" in City name]', cond = 'name LIKE "%천%"')

## 아래 조건식은 문자열에 "천"이 들어가지 않는 데이터들을 모두 출력해준다.
select('["천" not in City name]', cond = 'name not like "%천%"')

[query] select  * from tCity where name LIKE "%천%";

["천" in City name]
('순천', 910, 27, 'n', '전라')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  * from tCity where name not like "%천%";

["천" not in City name]
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('오산', 42, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')




In [26]:
## 이레 조건식은 천으로 시작하는 데이터만 출력한다.
select('["천"으로 시작되는 도시 이름]', cond = 'name like "천%"')

## 아래 조건식은 천으로 끝나는 데이터만 출력한다.
select('["천"으로 끝나는 도시 이름]', cond = 'name like "%천"')

[query] select  * from tCity where name like "천%";

["천"으로 시작되는 도시 이름]


[query] select  * from tCity where name like "%천";

["천"으로 끝나는 도시 이름]
('순천', 910, 27, 'n', '전라')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')




- 와일드 카드와 동일한 특수문자를 사용할 경우 escape 문자를 사용한다.
~~~ SQL
    LIKE '%30#%' ESCAPE '#'
~~~
    - 위 예시에서 %30#%는 30%를 포함하고 있는 문자열을 조회한다.
    
##### **연습 문제 #006.** 

> Q1. 직원 목록에서 '정'씨를 조사하라.   
> Q2. 이름에 '신'자가 포함된 직원을 조사하라.

In [27]:
select('[A1.]', table_name = 'tStaff', cond = 'name like "정%"')

select('[A2.]', table_name = 'tStaff', cond = 'name like "%신%"')

[query] select  * from tStaff where name like "정%";

[A1.]
('정몽주', '총무부', '남', datetime.date(2010, 9, 16), '대리', 370, Decimal('89.50'))
('정약용', '총무부', '남', datetime.date(2020, 3, 14), '과장', 380, Decimal('69.80'))


[query] select  * from tStaff where name like "%신%";

[A2.]
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 400, Decimal('92.00'))




### 2-5. BETWEEN
- BETWEEN A AND B 문은 최소값 A ~ 최대값 B 사이의 값을 조회해준다.
- 부등호를 사용하는 범위 조건은 수치값에 대해 사용하지만, between A and B는  
  문자열이나 날짜 등에도 사용할 수 있다.

In [28]:
select('[인구가 50 ~ 100만 사이의 값인 도시]', cond = 'popu between 50 and 100')

## 아래 조건문은 위 조건문과 같은 결과를 낸다.
select('[인구가 50 ~ 100만 사이의 값인 도시]', cond = 'popu >= 50 AND popu <= 100')

[query] select  * from tCity where popu between 50 and 100;

[인구가 50 ~ 100만 사이의 값인 도시]
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')


[query] select  * from tCity where popu >= 50 AND popu <= 100;

[인구가 50 ~ 100만 사이의 값인 도시]
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')




In [29]:
select('[이름이 가 ~ 사 사이인 직원들]', table_name = 'tStaff', cond = 'name between "가" and "사"')
select('[입사일이 2015.01.01 ~ 2018.01.01인 직원들]', table_name = 'tStaff', 
       cond = 'joindate between "20150101" and "20180101"')

[query] select  * from tStaff where name between "가" and "사";

[이름이 가 ~ 사 사이인 직원들]
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))


[query] select  * from tStaff where joindate between "20150101" and "20180101";

[입사일이 2015.01.01 ~ 2018.01.01인 직원들]
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 350, Decimal('71.25'))
('이율곡', '총무부', '남', datetime.date(2016, 3, 8), '과장', 385, Decimal('65.40'))




##### **연습 문제 #007.** 

> Q1. 면적 500 ~ 1,000 사이의 도시 목록을 조사하라.   
> Q2. 월급이 200만원대인 직원의 목록을 구하라.

In [30]:
select('[A1.]', cond = 'area between 500 and 1000')
select('[A2.]', cond = 'salary between 200 and 299', table_name = 'tStaff')

[query] select  * from tCity where area between 500 and 1000;

[A1.]
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('청주', 940, 83, 'n', '충청')


[query] select  * from tStaff where salary between 200 and 299;

[A2.]
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('허난설헌', '인사과', '여', datetime.date(2020, 1, 5), '사원', 285, Decimal('44.50'))
('황진이', '인사과', '여', datetime.date(2012, 5, 5), '사원', 275, Decimal('52.50'))




### 2-6. IN
- BETWEEN 문은 A와 B 사이 연속된 범위의 값만 검색할 수 있는 반면 IN 문은 불연속적인  
  값 여러 개의 목록을 제공하여 목록과 일치하는 레코드를 검색한다.
  
  - IN 뒤의 괄호 안에 콤마로 구분된 값 목록을 나열하여 그 중 하나에 해당하는지 점검한다.
  - 값 개수에는 제한이 없어 무수히 많은 값을 넣을 수도 있다.

In [31]:
## 아래 조건문은 지역이 경상도이거나 전라도인 데이터만 조회한다.
select('[지역이 "경상" 이거나 "전라"인 데이터]', cond = 'region in ("경상", "전라")')

## 아래 조건문은 위 조건문과 같은 결과 셋을 출력한다.
select('[지역이 "경상" 이거나 "전라"인 데이터]', cond = 'region = "경상" or region =  "전라"')

[query] select  * from tCity where region in ("경상", "전라");

[지역이 "경상" 이거나 "전라"인 데이터]
('부산', 765, 342, 'y', '경상')
('순천', 910, 27, 'n', '전라')
('전주', 205, 65, 'n', '전라')


[query] select  * from tCity where region = "경상" or region =  "전라";

[지역이 "경상" 이거나 "전라"인 데이터]
('부산', 765, 342, 'y', '경상')
('순천', 910, 27, 'n', '전라')
('전주', 205, 65, 'n', '전라')




In [32]:
## 아래 조건문은 지역이 경상도이나 전라도가 아닌 데이터만 조회한다.
select('[지역이 "경상" 이나 "전라"가 아닌 데이터]', cond = 'region not in ("경상", "전라")')

## LIKE 문과는 결합하여 사용할 수 없고 OR로 묶어야한다. 
## 위 방법은 복잡도가 증가하고 성능상의 문제가 발생할 수 있다.

## 아래 조건문은 안씨이거나 이씨인 직원을 뽑는다.
select('[이씨이거나 안씨인 직원]', cond = "name like '이%' or name like '안%'", 
        table_name = 'tStaff')

[query] select  * from tCity where region not in ("경상", "전라");

[지역이 "경상" 이나 "전라"가 아닌 데이터]
('서울', 605, 974, 'y', '경기')
('오산', 42, 21, 'n', '경기')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  * from tStaff where name like '이%' or name like '안%';

[이씨이거나 안씨인 직원]
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('이사부', '총무부', '남', datetime.date(2000, 2, 3), '대리', 375, Decimal('50.00'))
('이율곡', '총무부', '남', datetime.date(2016, 3, 8), '과장', 385, Decimal('65.40'))




##### **연습 문제 #008.** 

> Q1. 총무부나 영업부에 근무하는 직원의 목록을 조사하라.   
> Q2. 인사과나 영업부에 근무하는 대리의 목록을 조사하라.  
> Q3. 차장급 이상의 여직원 목록을 조사하라.

In [33]:
select('[A1].', table_name = 'tStaff', 
       cond = 'depart in ("총무부", "영업부")')
select('[A2].', table_name = 'tStaff', 
       cond = 'depart in ("인사과", "영업부") and grade = "대리"')
select('[A3].', table_name = 'tStaff', 
       cond = 'grade in ("차장", "부장", "이사") and gender = "여"')

[query] select  * from tStaff where depart in ("총무부", "영업부");

[A1].
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('김유신', '총무부', '남', datetime.date(2000, 2, 3), '이사', 420, Decimal('88.80'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('신사임당', '영업부', '여', datetime.date(2013, 6, 19), '부장', 400, Decimal('92.00'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('유관순', '영업부', '여', datetime.date(2009, 3, 1), '과장', 380, None)
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 350, Decimal('71.25'))
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), '사원', 330, None)
('이사부', '총무부', '남', datetime.date(2000, 2, 3), '대리', 375, Decimal('50.00'))
('이율곡', '총무부', '남', datetime.date(2016, 3, 8), '과장', 385, Decimal('65.40'))
('정몽주', '총무부', '남', datetime.date(2010, 9, 16), '대리', 370, Decimal('89.50'))
('정약용', '총무부', '남', datetime.date(2

## **3. 정렬**
### 3-1. ORDER BY
- SELECT 문은 명령에 별다른 명시가 없을 경우 DBMS의 기본 순서를 따른다.
    - ORACLE은 레코드의 입력 순서를 기억해 두고 출력한다.
    - SQL Server와 Maria DB는 기본키에 대해 오름차 순으로 정렬한다.
    
~~~ SQL
    ORDER BY 필드 [ASC | DESC]
~~~
- ASC는 오름차순으로 DESC는 내림차순으로 정렬하고, 기본값은 ASC이다.

In [34]:
## 아래 조건문은 인구수를 기준으로 정렬하여 인구수가 적은 도시부터 출력한다.
select('[POPU for asc]', order_column = 'popu', order = 'asc')

## 아래 조건문은 인구수를 기준으로 정렬하여 인구수가 많은 도시부터 출력한다.
select('[POPU for desc]', order_column = 'popu', order = 'desc')

[query] select  * from tCity order by popu asc;

[POPU for asc]
('홍천', 1819, 7, 'n', '강원')
('오산', 42, 21, 'n', '경기')
('순천', 910, 27, 'n', '전라')
('춘천', 1116, 27, 'n', '강원')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('부산', 765, 342, 'y', '경상')
('서울', 605, 974, 'y', '경기')


[query] select  * from tCity order by popu desc;

[POPU for desc]
('서울', 605, 974, 'y', '경기')
('부산', 765, 342, 'y', '경상')
('청주', 940, 83, 'n', '충청')
('전주', 205, 65, 'n', '전라')
('순천', 910, 27, 'n', '전라')
('춘천', 1116, 27, 'n', '강원')
('오산', 42, 21, 'n', '경기')
('홍천', 1819, 7, 'n', '강원')




- order by 문에 두 개 이상의 기준 필드를 지정할 수 있다.
    - 첫 번째 기준 필드의 값이 같은 경우 두 번째 기준 필드로 정렬한다.

In [35]:
## 아래 조건문은 지역별로 정렬하되 같은 지역에 속한 도시끼리는 이름의 내림차 순으로 정렬한다.
## region에 대해서는 order 기준이 지정되어 있지 않으므로 기본 값인 asc가 적용된다.
select(text         = '[지역은 오름차순, 같은 지역의 도시끼리는 내림차순으로 정렬]',
       column       =                      'region, name, area, popu',
       order_column =                                  'region, name',
       order        =                                          'desc'
      )

[query] select  region, name, area, popu from tCity order by region, name desc;

[지역은 오름차순, 같은 지역의 도시끼리는 내림차순으로 정렬]
('강원', '홍천', 1819, 7)
('강원', '춘천', 1116, 27)
('경기', '오산', 42, 21)
('경기', '서울', 605, 974)
('경상', '부산', 765, 342)
('전라', '전주', 205, 65)
('전라', '순천', 910, 27)
('충청', '청주', 940, 83)




- ORDER BY는 보통 필드명으로도 지정하지만, 각 필드의 인덱스 값으로도 지정할 수 있다.
    - 단, SQL에서는 인덱스가 1번부터 시작한다.
    
- WHERE 절과 함께 사용할때는 ORDER BY가 WHERE보다 뒤에 있어야 한다.

In [36]:
## 아래 쿼리문은 SELECT * FROM tCity ORDER BY area;와 동일한 결과 셋을 반환한다.
select('[ORDER BY INDEX 2]', order_column = '2')

## 아래 쿼리문은 도시 이름과 인구밀도를 반환한다.
select('[이름과 인구 밀도만 표시하는데, 인구밀도를 기준으로 표시]', column = 'name, popu*10000/area'
       , order_column = 'popu*10000/area')


## 아래 쿼리문은 경기 지역인 도시들 중에서 면적순으로 정렬한 결과를 반환한다.
select('[경기도면서, area 오름차순 정렬]', cond = 'region="경기"', order_column = 'area')

[query] select  * from tCity order by 2 asc;

[ORDER BY INDEX 2]
('오산', 42, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')
('서울', 605, 974, 'y', '경기')
('부산', 765, 342, 'y', '경상')
('순천', 910, 27, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')


[query] select  name, popu*10000/area from tCity order by popu*10000/area asc;

[이름과 인구 밀도만 표시하는데, 인구밀도를 기준으로 표시]
('홍천', Decimal('38.4827'))
('춘천', Decimal('241.9355'))
('순천', Decimal('296.7033'))
('청주', Decimal('882.9787'))
('전주', Decimal('3170.7317'))
('부산', Decimal('4470.5882'))
('오산', Decimal('5000.0000'))
('서울', Decimal('16099.1736'))


[query] select  * from tCity where region="경기"order by area asc;

[경기도면서, area 오름차순 정렬]
('오산', 42, 21, 'n', '경기')
('서울', 605, 974, 'y', '경기')




##### **연습 문제 #009.** 

> Q1. 직원 목록을 월급이 적은 사람부터 순서대로 정렬하되 월급이 같다면 성취도가 높은 사람을 먼저 출력하라  
> Q2. 영업부 직원을 먼저 입사한 순서대로 정렬하라.

In [37]:
select('[A1.]'                       , table_name   = 'tStaff', 
       order_column = 'salary, score', order        = 'desc')

select('[A2.]'               , table_name   = 'tStaff',
       cond = 'depart="영업부"', order_column = 'joindate')

[query] select  * from tStaff order by salary, score desc;

[A1.]
('안중근', '인사과', '남', datetime.date(2012, 5, 5), '대리', 256, Decimal('76.50'))
('황진이', '인사과', '여', datetime.date(2012, 5, 5), '사원', 275, Decimal('52.50'))
('성삼문', '영업부', '남', datetime.date(2014, 6, 8), '대리', 285, Decimal('87.75'))
('허난설헌', '인사과', '여', datetime.date(2020, 1, 5), '사원', 285, Decimal('44.50'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), '사원', 330, None)
('논개', '인사과', '여', datetime.date(2010, 9, 16), '대리', 340, Decimal('46.20'))
('윤봉길', '영업부', '남', datetime.date(2015, 8, 15), '과장', 350, Decimal('71.25'))
('정몽주', '총무부', '남', datetime.date(2010, 9, 16), '대리', 370, Decimal('89.50'))
('안창호', '영업부', '남', datetime.date(2015, 8, 15), '사원', 370, Decimal('74.20'))
('이사부', '총무부', '남', dateti

### 3-2. DISTINCT
- 테이블에 데이터가 많아지는 경우에 SELECT 하는 것만으로도 시간이 오래걸린다.
    - 이러한 문제를 해소하기 위해 중복된 데이터는 출력하지 않는 등 여러 방법을 사용한다.
    - 중복된 값을 제거할 때는 DISTINCT를 사용한다.
    
    ~~~ SQL
        SELECT DISTINC * FROM [테이블 명]
    ~~~

In [38]:
select('[all data showing even 중복]', column = 'region')

## 아래 쿼리문은 중복된 데이터는 한 번만 보여준다.
select('[without 중복 데이터]', column = 'region', distinct = True)

## 아래 쿼리문은 중복된 데이터는 제거하고, 지역 이름을 오름차순으로 정렬한다.
select('[without 중복 데이터, region asc]', column = 'region', distinct = True,
        order_column = 'region')

[query] select  region from tCity ;

[all data showing even 중복]
('경상',)
('경기',)
('전라',)
('경기',)
('전라',)
('충청',)
('강원',)
('강원',)


[query] select distinct region from tCity ;

[without 중복 데이터]
('경상',)
('경기',)
('전라',)
('충청',)
('강원',)


[query] select distinct region from tCity order by region asc;

[without 중복 데이터, region asc]
('강원',)
('경기',)
('경상',)
('전라',)
('충청',)




##### **연습 문제 #010.** 

> Q1. 2020년 이후 신입 사원을 받은 적 있는 부서 목록을 조사하라

In [39]:
select('[A1.]'          , distinct = True, table_name = 'tStaff',
       column = 'depart', cond     = 'joindate >= 20200101')

[query] select distinct depart from tStaff where joindate >= 20200101;

[A1.]
('총무부',)
('인사과',)




### **3-3. LIMIT**
- LIMIT 문은 테이블 내의 k개의 레코드를 뽑아내준다.
    - 오라클에서는 의사 컬럼, SQL Server에서는 TOP문을 사용한다.
    ~~~SQL
        - SQL Server : SELECT TOP k [필드명] FROM [테이블 명];
        - Maria DB   : SELECT [필드명] FROM [테이블 명] LIMIT [건너뛸 개수], k;
    ~~~
    - 건너뛸 개수를 비워두면 기본 값은 0으로 적용되어 첫 행부터 출력된다.

In [40]:
## 아래 쿼리문은 면적이 넓은 상위 4개의 도시
select('[면적이 넓은 상위 4개 도시]', order_column = 'area', 
       order = 'desc', limit_k = 4)

## 아래 쿼리문은 다음 구문 2개는 건너뛰고, 이후 3개의 행을 보여준다.
select('[맨 앞 2개 건너뛰고, 그 다음 3개]', limit_k = '2, 3')

## 아래 쿼리문은 위의 쿼리문과 동일한 결과 셋을 출력한다.
print('[맨 앞 2개 건너뛰고, 그 다음 3개]')
cursor.execute('select * from tCity order by area desc limit 3 offset 2')
for result in cursor.fetchall(): print(f'{result}')
print('\n')

[query] select  * from tCity order by area desc limit 4;

[면적이 넓은 상위 4개 도시]
('홍천', 1819, 7, 'n', '강원')
('춘천', 1116, 27, 'n', '강원')
('청주', 940, 83, 'n', '충청')
('순천', 910, 27, 'n', '전라')


[query] select  * from tCity  limit 2, 3;

[맨 앞 2개 건너뛰고, 그 다음 3개]
('순천', 910, 27, 'n', '전라')
('오산', 42, 21, 'n', '경기')
('전주', 205, 65, 'n', '전라')


[맨 앞 2개 건너뛰고, 그 다음 3개]
('청주', 940, 83, 'n', '충청')
('순천', 910, 27, 'n', '전라')
('부산', 765, 342, 'y', '경상')




##### **연습 문제 #010.** 

> Q1. 직원을 월급순으로 정렬한 후 12위에서 16위까지 출력하라.

In [41]:
select('[A1.]', table_name = 'tStaff', order_column = 'salary',
        limit_k = '12, 5', order = 'desc')

[query] select  * from tStaff order by salary desc limit 12, 5;

[A1.]
('을지문덕', '영업부', '남', datetime.date(2019, 6, 29), '사원', 330, None)
('강감찬', '영업부', '남', datetime.date(2018, 10, 9), '사원', 320, Decimal('56.00'))
('선덕여왕', '인사과', '여', datetime.date(2017, 8, 3), '사원', 315, Decimal('45.10'))
('대조영', '총무부', '남', datetime.date(2020, 7, 7), '차장', 290, Decimal('49.90'))
('허난설헌', '인사과', '여', datetime.date(2020, 1, 5), '사원', 285, Decimal('44.50'))




In [42]:
conn.commit()
conn.close()

# **99. 참고자료**
## **99-1. 도서** 
- 소문난 명강의 - 김상형의 SQL 정복 | 김상형 저 / 한빛 미디어

## **99-2. 논문, 학술지**

## **99-3. 웹 사이트**

## **99-4. 데이터셋 출처**
